In [1]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


### just try 17 feature to predict next pm2.5,

In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [3]:
print('train_data shape: ',train_data.shape)
print('test_data shape: ',test_data.shape)

train_data shape:  (4320, 27)
test_data shape:  (4320, 11)


In [4]:
train_data1=train_data.replace('NR',0)
train_data1=train_data1.iloc[:,3:]

In [5]:
year_data={}
for m in range(12):
    month_data=np.empty((18,20*24))
    for d in range(20):
        month_data[:,d*24:(d+1)*24]=train_data1.iloc[m*20*18+d*18:m*20*18+(d+1)*18,:]
    year_data[m]=month_data

In [6]:
year_data1=year_data.copy()

In [7]:
#### 台湾温度不会低于0度

In [8]:
# temperature always >=0 in Taiwan and change fast when temperature <=0 ,so correct it
for idx in [0]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <=5:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >5:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
---------- pm2.5 start : 
 -3.1 

-8.0
-7.2
-6.8
-6.5
-7.1
-7.4
-8.1
-8.3
-8.4
-9.3
-10.6
-11.2
-12.1
0.0
0.0
0.0
0.0
-12.3
pm2.5 end ------------
correct value between  : 19.0 18.0
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11


In [9]:
min(478,480)

478

In [10]:
for idx in [1, 2, 4, 6, 9, 12, 13]:
    for m in range(12):
        print(' month : ', m)
        i = 0
        while i < 480:
            if year_data1[m][idx, i] <= 0:
                print('---------- pm2.5 start : \n', year_data1[m][idx, i],
                      '\n')
                for k in range(30):
                    if k + i + 1 > 479:
                        break
                    else:
                        if year_data1[m][idx, k + i + 1] > 0:
                            break
                        else:
                            print(year_data1[m][idx, k + i + 1])
                i = i + k
                print('pm2.5 end ------------')
                print(idx,i-1-k,i+1)
                if year_data1[m][idx, min(i + 1,479)]==0:
                    year_data1[m][idx, min(i + 1,479)]=1
                if i+1>=480:
                    print('correct value between  :',year_data1[m][idx, i - 1 - k], year_data1[m][idx, i - 1 - k])
#                     year_data1[m][idx, 479]=year_data1[m][idx, i - 1 - k]
                    year_data1[m][idx,i - k:] = year_data1[m][idx, i - 1 - k]
                else:
                    print('correct value between  :',year_data1[m][idx, i - 1 - k], year_data1[m][idx, i + 1])
                ## add correct to mean value
                    year_data1[m][idx,i - k:i + 1] = (year_data1[m][idx, i - 1 - k] +year_data1[m][idx, i + 1]) / 2
            i = i + 1

 month :  0
 month :  1
 month :  2
 month :  3
---------- pm2.5 start : 
 0.0 

0.0
pm2.5 end ------------
1 177 180
correct value between  : 1.6 1.6
 month :  4
 month :  5
---------- pm2.5 start : 
 0.0 

pm2.5 end ------------
1 370 372
correct value between  : 1.6 1.6
 month :  6
 month :  7
---------- pm2.5 start : 
 0.0 

pm2.5 end ------------
1 179 181
correct value between  : 1.6 1.7
---------- pm2.5 start : 
 0.0 

pm2.5 end ------------
1 442 444
correct value between  : 1.7 1.6
 month :  8
---------- pm2.5 start : 
 0.0 

pm2.5 end ------------
1 105 107
correct value between  : 1.7 1.8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.2 

-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
pm2.5 end ------------
1 73 83
correct value between  : 1.8 1.8
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.12 

pm2.5 end ------------
2 73 75
corre

correct value between  : 1.8 2.0
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.2 

-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
pm2.5 end ------------
13 73 83
correct value between  : 2.0 1.9


In [11]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [12]:
from scipy.stats import boxcox,boxcox_normmax
from scipy.special import boxcox1p

In [13]:
all_train_data2=pd.DataFrame()
for i in range(all_train_data1.shape[1]):
    if i != 9:
        lm=boxcox_normmax(all_train_data1[i]+1)
        all_train_data2[i]=boxcox1p(all_train_data1[i],lm)
    else:
        all_train_data2[i]=np.sqrt(all_train_data1[i])
        
    
    print(all_train_data1.index[i],'--skew : ',all_train_data1[i].skew(),'--kurt : ',all_train_data1[i].kurt())
    print('lambda : ',lm)
    print(all_train_data2.index[i],'--skew : ',all_train_data2[i].skew(),'--kurt : ',all_train_data2[i].kurt())
    print('--------------------------------------------------------------------')

0 --skew :  -0.2834134242907861 --kurt :  -0.7968009844193018
lambda :  1.461645538288768
0 --skew :  -0.07414275326700835 --kurt :  -0.9258821506244836
--------------------------------------------------------------------
1 --skew :  0.012533992778003633 --kurt :  -0.587252860997554
lambda :  0.8920982850249821
1 --skew :  0.005496794798210646 --kurt :  -0.5911686671270657
--------------------------------------------------------------------
2 --skew :  9.165303973318487 --kurt :  132.25692190213024
lambda :  -3.259330902857795
2 --skew :  -0.003907562716963148 --kurt :  -0.06972890461562153
--------------------------------------------------------------------
3 --skew :  2.172048918138678 --kurt :  8.458548044518952
lambda :  -6.182620739009405
3 --skew :  0.05859262861967747 --kurt :  -0.5568735662256521
--------------------------------------------------------------------
4 --skew :  5.381555433985442 --kurt :  41.286157239228515
lambda :  -0.8099547985075456
4 --skew :  -0.02884493154

In [14]:
# all_train_data3=pd.DataFrame()
# for i in range(all_train_data2.shape[1]):
#     if i!=9:
#         all_train_data3[i]=(all_train_data2[i] - all_train_data2[i].mean()) / all_train_data2[i].std()
#     else:
#         all_train_data3[i]=all_train_data2[i]

In [15]:
all_train_data2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,70.426493,1.604398,0.186377,0.079806,0.664179,2.283694,1.937404,7.449352,9.327724,4.334282,0.008977,15482.226219,1.041644,0.240173,12.803597,11.687312,1.046624,0.813470
std,25.536335,0.075221,0.041562,0.032614,0.170238,0.523410,0.306804,2.431622,3.291614,1.716322,0.035324,6063.422584,0.296484,0.000672,3.602422,3.020925,0.263948,0.292340
min,12.833239,1.417616,0.068067,0.000000,0.091724,0.000000,0.769032,0.000000,0.000000,1.000000,0.000000,1616.845054,0.094142,0.237929,0.096879,0.000000,0.000000,0.000000
25%,49.926550,1.508009,0.158556,0.055291,0.557687,1.938097,1.735287,5.766358,7.204513,3.000000,0.000000,10617.557930,0.846343,0.239463,9.751980,9.184518,0.857234,0.592611
50%,70.525274,1.598028,0.188618,0.076901,0.665221,2.270782,1.916995,7.472206,9.283713,4.242641,0.000000,15534.166404,1.026800,0.240036,12.099014,11.107693,1.042283,0.818152
75%,88.521007,1.687688,0.215417,0.103613,0.765212,2.651342,2.143833,9.063998,11.523345,5.477226,0.000000,20398.022735,1.230209,0.240513,15.383128,13.963600,1.233421,1.030980
max,133.380091,1.865986,0.306532,0.160805,1.160089,3.876332,2.905076,20.678088,20.721569,10.583005,0.161645,30295.187454,2.146147,0.242770,18.965279,16.819218,1.852950,1.618322


In [16]:
all_train_data3=pd.DataFrame()
for i in range(all_train_data2.shape[1]):
        all_train_data3[i]=(all_train_data2[i] - all_train_data2[i].mean()) / all_train_data2[i].std()

In [17]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.00000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.610382,1.707231,0.388436,0.140427,2.137483,10.125990,12.254115,31.905469,42.709201,21.731250,0.200625,73.229167,2.76967,1.845304,156.329271,158.482795,2.297240,1.712760
std,6.062216,0.083752,0.323505,0.104645,2.281195,6.187555,7.571422,18.703486,26.222292,16.361366,2.045443,13.361351,1.80898,0.149816,95.745881,94.697432,1.065408,1.062683
min,6.700000,1.500000,0.080000,0.000000,0.100000,0.000000,1.300000,0.000000,0.000000,1.000000,0.000000,29.000000,0.10000,1.500000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.60000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.30000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,30.000000,0.000000,84.000000,3.40000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.00000,3.000000,360.000000,360.000000,7.700000,7.000000


In [18]:
all_train_data2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,70.426493,1.604398,0.186377,0.079806,0.664179,2.283694,1.937404,7.449352,9.327724,4.334282,0.008977,15482.226219,1.041644,0.240173,12.803597,11.687312,1.046624,0.813470
std,25.536335,0.075221,0.041562,0.032614,0.170238,0.523410,0.306804,2.431622,3.291614,1.716322,0.035324,6063.422584,0.296484,0.000672,3.602422,3.020925,0.263948,0.292340
min,12.833239,1.417616,0.068067,0.000000,0.091724,0.000000,0.769032,0.000000,0.000000,1.000000,0.000000,1616.845054,0.094142,0.237929,0.096879,0.000000,0.000000,0.000000
25%,49.926550,1.508009,0.158556,0.055291,0.557687,1.938097,1.735287,5.766358,7.204513,3.000000,0.000000,10617.557930,0.846343,0.239463,9.751980,9.184518,0.857234,0.592611
50%,70.525274,1.598028,0.188618,0.076901,0.665221,2.270782,1.916995,7.472206,9.283713,4.242641,0.000000,15534.166404,1.026800,0.240036,12.099014,11.107693,1.042283,0.818152
75%,88.521007,1.687688,0.215417,0.103613,0.765212,2.651342,2.143833,9.063998,11.523345,5.477226,0.000000,20398.022735,1.230209,0.240513,15.383128,13.963600,1.233421,1.030980
max,133.380091,1.865986,0.306532,0.160805,1.160089,3.876332,2.905076,20.678088,20.721569,10.583005,0.161645,30295.187454,2.146147,0.242770,18.965279,16.819218,1.852950,1.618322


In [19]:
all_train_data3.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03
mean,1.019875e-13,-4.451758e-13,6.653925e-14,2.549689e-14,1.782786e-14,-6.727368e-14,-4.221841e-14,2.062856e-15,5.215490e-14,-1.005904e-14,-1.659610e-15,6.245795e-14,3.845739e-14,-6.354188e-12,1.616595e-14,-8.086471e-15,3.354384e-14,3.181464e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.255345e+00,-2.483110e+00,-2.846587e+00,-2.447008e+00,-3.362674e+00,-4.363104e+00,-3.808196e+00,-3.063532e+00,-2.833784e+00,-1.942691e+00,-2.541260e-01,-2.286725e+00,-3.195798e+00,-3.340700e+00,-3.527271e+00,-3.868786e+00,-3.965265e+00,-2.782619e+00
25%,-8.027755e-01,-1.281405e+00,-6.693840e-01,-7.516859e-01,-6.255434e-01,-6.602794e-01,-6.587790e-01,-6.921282e-01,-6.450364e-01,-7.774080e-01,-2.541260e-01,-8.022974e-01,-6.587225e-01,-1.057126e+00,-8.471015e-01,-8.284860e-01,-7.175293e-01,-7.554874e-01
50%,3.868283e-03,-8.467753e-02,5.390355e-02,-8.905258e-02,6.122491e-03,-2.466757e-02,-6.652185e-02,9.398760e-03,-1.337080e-02,-5.339410e-02,-2.541260e-01,8.566150e-03,-5.006604e-02,-2.047486e-01,-1.955858e-01,-1.918681e-01,-1.644871e-02,1.601343e-02
75%,7.085792e-01,1.107276e+00,6.987095e-01,7.299705e-01,5.934813e-01,7.024097e-01,6.728357e-01,6.640201e-01,6.670347e-01,6.659262e-01,-2.541260e-01,8.107297e-01,6.360047e-01,5.054627e-01,7.160547e-01,7.535070e-01,7.077023e-01,7.440291e-01
max,2.465256e+00,3.477601e+00,2.890982e+00,2.483615e+00,2.913043e+00,3.042810e+00,3.154037e+00,5.440292e+00,3.461476e+00,3.640765e+00,4.321989e+00,2.443003e+00,3.725344e+00,3.864731e+00,1.710428e+00,1.698787e+00,3.054866e+00,2.753135e+00


In [20]:
from scipy import stats
import seaborn as sns

In [21]:
stats.probplot(all_train_data1[9].values,fit=stats.norm,plot=plt)
plt.show()

In [22]:
sns.distplot(np.sqrt(all_train_data1[9].values),fit=stats.norm)
plt.show()

In [72]:
stats.probplot(np.sqrt(all_train_data1[9].values),fit=stats.norm,plot=plt)
plt.show()

In [24]:
stats.probplot(all_train_data2[9].values,fit=stats.norm,plot=plt)
plt.show()

In [25]:

sns.distplot(all_train_data2[9].values,fit=stats.norm)

In [26]:
stats.probplot(all_train_data3[9].values,fit=stats.norm,plot=plt)

((array([-3.67199794, -3.43886226, -3.31047894, ...,  3.31047894,
          3.43886226,  3.67199794]),
  array([-1.9426908 , -1.9426908 , -1.9426908 , ...,  3.55762257,
          3.55762257,  3.64076476])),
 (0.9945437850375948, -9.39409242307104e-15, 0.9940455917244347))

In [27]:
sns.distplot(all_train_data3[9].values,fit=stats.norm)

- all_train_data1 : fix outlier 
- all_train_data11 : fix outlier and sqrt pm2.5
- all_train_data2 : fix skew and kurt
- all_train_data3 : 18 feats norm

In [28]:
8897+989

9886

### just use original data to predict next pm2.5
- use all_train_data1

In [29]:
year_data3={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data1.T.iloc[:,m*480:(m+1)*480]
    year_data3[m]=month_data3

- x shape : 12 * 479 (480-1,since next pm2.5) , 17
- y shape : t+1 pm2.5 ,12 * 479

In [30]:
year_data3[0].shape

(18, 480)

In [31]:
year_data3[m].transpose().shape

(480, 18)

In [32]:
year_data3[0].transpose()[:479,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]][0]

array([14.  ,  1.8 ,  0.51,  0.2 ,  0.9 , 16.  , 17.  , 16.  , 56.  ,
       26.  ,  0.  , 77.  ,  1.8 ,  2.  , 37.  , 35.  ,  1.4 ,  0.5 ])

In [33]:
year_data3[m].transpose()[:5,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]

array([[2.3e+01, 1.6e+00, 2.2e-01, 7.0e-02, 1.5e+00, 6.1e+00, 7.6e+00,
        1.6e+01, 3.4e+01, 2.2e+01, 0.0e+00, 7.9e+01, 1.8e+00, 1.7e+00,
        9.0e+01, 9.3e+01, 1.8e+00, 1.3e+00],
       [2.3e+01, 1.7e+00, 2.0e-01, 7.0e-02, 1.3e+00, 6.1e+00, 7.4e+00,
        1.8e+01, 1.9e+01, 1.7e+01, 0.0e+00, 7.8e+01, 1.6e+00, 1.7e+00,
        4.9e+01, 5.0e+01, 2.1e+00, 9.0e-01],
       [2.3e+01, 1.7e+00, 1.8e-01, 3.0e-02, 1.4e+00, 4.6e+00, 6.0e+00,
        2.7e+01, 2.4e+01, 1.7e+01, 0.0e+00, 8.1e+01, 2.5e+00, 1.7e+00,
        8.2e+01, 9.9e+01, 3.2e+00, 1.0e+00],
       [2.2e+01, 1.7e+00, 3.0e-01, 6.0e-02, 1.1e+00, 6.0e+00, 7.0e+00,
        3.3e+01, 2.9e+01, 2.2e+01, 0.0e+00, 8.2e+01, 3.2e+00, 1.7e+00,
        8.5e+01, 7.1e+01, 4.0e+00, 2.2e+00],
       [2.1e+01, 1.7e+00, 4.0e-01, 5.0e-02, 9.0e-01, 6.3e+00, 7.1e+00,
        2.2e+01, 3.5e+01, 2.7e+01, 0.0e+00, 7.9e+01, 1.5e+00, 1.8e+00,
        5.8e+01, 5.8e+01, 3.3e+00, 3.7e+00]])

In [34]:
year_data3[0].transpose()[1:,9][:5]

array([39., 36., 35., 31., 28.])

In [35]:
x=np.empty((12*479,18))
y=np.empty((12*479,1))

In [36]:
for m in range(12):
    x[m*479:(m+1)*479,:]=year_data3[m].transpose()[:479,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
    y[m*479:(m+1)*479,:]=year_data3[m].transpose()[1:,9:10]

In [37]:
x[:5]

array([[ 14.  ,   1.8 ,   0.51,   0.2 ,   0.9 ,  16.  ,  17.  ,  16.  ,
         56.  ,  26.  ,   0.  ,  77.  ,   1.8 ,   2.  ,  37.  ,  35.  ,
          1.4 ,   0.5 ],
       [ 14.  ,   1.8 ,   0.41,   0.15,   0.6 ,   9.2 ,   9.8 ,  30.  ,
         50.  ,  39.  ,   0.  ,  68.  ,   2.  ,   2.  ,  80.  ,  79.  ,
          1.8 ,   0.9 ],
       [ 14.  ,   1.8 ,   0.39,   0.13,   0.5 ,   8.2 ,   8.7 ,  27.  ,
         48.  ,  36.  ,   0.  ,  67.  ,   1.7 ,   2.  ,  57.  ,   2.4 ,
          1.  ,   0.6 ],
       [ 13.  ,   1.8 ,   0.37,   0.12,   1.7 ,   6.9 ,   8.6 ,  23.  ,
         35.  ,  35.  ,   0.  ,  74.  ,   1.6 ,   1.9 ,  76.  ,  55.  ,
          0.6 ,   0.3 ],
       [ 12.  ,   1.8 ,   0.35,   0.11,   1.8 ,   6.8 ,   8.5 ,  24.  ,
         25.  ,  31.  ,   0.  ,  72.  ,   1.9 ,   1.9 , 110.  ,  94.  ,
          1.7 ,   0.6 ]])

In [38]:
y[:5]

array([[39.],
       [36.],
       [35.],
       [31.],
       [28.]])

- x , y   : fixed outlier
    - x1,y1   : sqrt pm2.5
        - x12,y12 : norm
- x2,y2   : skew and kurt
- x3,y3   : norm

In [65]:
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [40]:
n_folds=5
for i in range(9):
    print('-----------------------------------',i,'----------------------------------------')
    lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x)
    rmse=np.sqrt(-cross_val_score(lasso,x,y,scoring='neg_mean_squared_error',cv=kf))
    score=rmse
    print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


----------------------------------- 0 ----------------------------------------


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa


Lasso score: 6.1767 (0.6346)

----------------------------------- 1 ----------------------------------------


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



Lasso score: 6.1767 (0.6346)

----------------------------------- 2 ----------------------------------------


...

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa


Lasso score: 6.1767 (0.6346)

----------------------------------- 7 ----------------------------------------


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



Lasso score: 6.1767 (0.6346)

----------------------------------- 8 ----------------------------------------

Lasso score: 6.1767 (0.6346)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


#### ust sqrt of  pm2.5

In [41]:
year_data33={}
all_train_data11=all_train_data1.copy()
## ust sqrt to pm2.5
all_train_data11[9]=np.sqrt(all_train_data11[9])
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data11.T.iloc[:,m*480:(m+1)*480]
    year_data33[m]=month_data3

In [42]:
x1=np.empty((12*479,18))
y1=np.empty((12*479,1))

In [43]:
for m in range(12):
    x1[m*479:(m+1)*479,:]=year_data33[m].transpose()[:479,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
    y1[m*479:(m+1)*479,:]=year_data33[m].transpose()[1:,9:10]

In [44]:
n_folds=5
for i in range(9):
    print('-----------------------------------',i,'----------------------------------------')
    lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x1)
    rmse=np.sqrt(-cross_val_score(lasso,x1,y1,scoring='neg_mean_squared_error',cv=kf))
    score=rmse
    print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


----------------------------------- 0 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 1 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 2 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 3 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 4 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 5 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 6 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 7 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 8 ----------------------------------------

Lasso score: 0.6969 (0.0394)



- x12,y12 : outlier ,sqrt pm2.5, norm

In [70]:
x12=pd.DataFrame(x1)
x12=x12.apply(lambda x: (x-x.mean()) / x.std())
x12=x12.values

In [71]:
n_folds=5
for i in range(9):
    print('-----------------------------------',i,'----------------------------------------')
    lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x12)
    rmse=np.sqrt(-cross_val_score(lasso,x12,y1,scoring='neg_mean_squared_error',cv=kf))
    score=rmse
    print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


----------------------------------- 0 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 1 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 2 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 3 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 4 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 5 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 6 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 7 ----------------------------------------

Lasso score: 0.6969 (0.0394)

----------------------------------- 8 ----------------------------------------

Lasso score: 0.6969 (0.0394)



### just use original data to predict next pm2.5
- use all_train_data2 & use all_train_data3

In [45]:
year_data4={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data2.T.iloc[:,m*480:(m+1)*480]
    year_data4[m]=month_data3

In [46]:
year_data5={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data3.T.iloc[:,m*480:(m+1)*480]
    year_data5[m]=month_data3

In [47]:
x2=np.empty((12*479,18))
y2=np.empty((12*479,1))

In [48]:
x3=np.empty((12*479,18))
y3=np.empty((12*479,1))

In [49]:
for m in range(12):
    x2[m*479:(m+1)*479,:]=year_data4[m].transpose()[:479,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
    y2[m*479:(m+1)*479,:]=year_data4[m].transpose()[1:,9:10]

In [50]:
for m in range(12):
    x3[m*479:(m+1)*479,:]=year_data5[m].transpose()[:479,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
    y3[m*479:(m+1)*479,:]=year_data5[m].transpose()[1:,9:10]

### use  all_train_data2

In [51]:
n_folds=5
print('-----------------------------------','start','----------------------------------------')
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x2)
rmse=np.sqrt(-cross_val_score(lasso,x2,y2,scoring='neg_mean_squared_error',cv=kf))
score=rmse
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


----------------------------------- start ----------------------------------------

Lasso score: 0.6953 (0.0384)



### use  all_train_data3

In [52]:
n_folds=5
print('-----------------------------------','start','----------------------------------------')
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x3)
rmse=np.sqrt(-cross_val_score(lasso,x3,y3,scoring='neg_mean_squared_error',cv=kf))
score=rmse
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

----------------------------------- start ----------------------------------------

Lasso score: 0.4050 (0.0222)



### x use all_train_data3 and y use all_train_data1

In [53]:
n_folds=5
print('-----------------------------------','start','----------------------------------------')
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x3)
rmse=np.sqrt(-cross_val_score(lasso,x3,y,scoring='neg_mean_squared_error',cv=kf))
score=rmse
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

----------------------------------- start ----------------------------------------

Lasso score: 7.1592 (0.7185)



In [54]:
lasso.fit(x3,y3)
y3_hat=lasso.predict(x3).reshape(-1,1)

In [55]:
lasso.fit(x2,y2)
y2_hat=lasso.predict(x2).reshape(-1,1)

In [56]:
lasso.fit(x,y)
y_hat=lasso.predict(x2).reshape(-1,1)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [57]:
x4=np.empty((12*479,3))
x4[:,0:1]=y_hat
x4[:,1:2]=y2_hat
x4[:,2:3]=y3_hat

In [58]:
x4[:5]

array([[-407.12615151,    5.05920993,    0.42210318],
       [-300.55996841,    5.93881557,    0.93289554],
       [-290.18052625,    5.65313117,    0.76514916],
       [-369.36139029,    5.55579873,    0.7104485 ],
       [-346.09756532,    5.24326854,    0.52891552]])

In [59]:
y3[:5]

array([[1.11326216],
       [0.97051618],
       [0.92162079],
       [0.71867781],
       [0.55771628]])

In [60]:
y2[:5]

array([[6.244998  ],
       [6.        ],
       [5.91607978],
       [5.56776436],
       [5.29150262]])

In [61]:
n_folds=5
print('-----------------------------------','start','----------------------------------------')
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x4)
rmse=np.sqrt(-cross_val_score(lasso,x4,y3,scoring='neg_mean_squared_error',cv=kf))
score=rmse
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

----------------------------------- start ----------------------------------------

Lasso score: 0.4022 (0.0212)



#### split x,y to y>2 and y<=2

In [93]:
x[np.where(y<=2)]

array([12. , 12. , 13. , 13. , 13. , 12. , 13. , 18. , 15. , 16. , 16. ,
       13. , 11. ,  7.7,  7.2,  9.4,  9.4,  9.6,  9.9, 12. , 14. , 13. ,
       11. , 11. , 11. , 11. , 11. , 13. , 14. , 13. , 18. , 17. , 16. ,
       16. , 16. , 16. , 17. , 14. , 14. , 14. , 14. , 15. , 15. , 21. ,
       14. , 14. , 14. , 14. , 14. , 14. , 13. , 13. , 12. , 12. , 13. ,
       12. , 12. , 14. , 14. , 14. , 18. , 19. , 19. , 18. , 19. , 19. ,
       26. , 23. , 24. , 22. , 19. , 19. , 18. , 18. , 18. , 22. , 22. ,
       24. , 24. , 24. , 26. , 27. , 26. , 24. , 26. , 30. , 29. , 25. ,
       25. , 24. , 24. , 24. , 24. , 24. , 24. , 24. , 23. , 31. , 29. ,
       24. , 25. , 26. , 24. , 24. , 25. , 26. , 28. , 28. , 28. , 27. ,
       26. , 24. , 24. , 24. , 24. , 26. , 26. , 26. , 26. , 26. , 25. ,
       28. , 26. , 25. , 26. , 28. , 27. , 27. , 27. , 30. , 28. , 28. ,
       29. , 27. , 27. , 27. , 27. , 27. , 27. , 28. , 32. , 27. , 28. ,
       28. , 26. , 27. , 25. , 25. , 26. , 27. , 27

In [94]:
x[np.where(y<=2)[0]]

array([[12.  ,  1.8 ,  0.3 , ..., 70.  ,  2.4 ,  1.8 ],
       [12.  ,  1.8 ,  0.29, ..., 64.  ,  2.4 ,  1.9 ],
       [13.  ,  1.8 ,  0.25, ..., 62.  ,  2.3 ,  2.  ],
       ...,
       [14.  ,  1.7 ,  0.21, ..., 83.  ,  3.7 ,  2.  ],
       [17.  ,  1.7 ,  0.24, ..., 58.  ,  3.1 ,  2.7 ],
       [18.  ,  1.7 ,  0.23, ..., 75.  ,  2.8 ,  2.8 ]])

In [95]:
n_folds=5
for i in range(9):
    print('-----------------------------------',i,'----------------------------------------')
    lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x1[np.where(y1<=2)[0]])
    rmse=np.sqrt(-cross_val_score(lasso,x1[np.where(y1<=2)[0]],y1[np.where(y1<=2)[0]],scoring='neg_mean_squared_error',cv=kf))
    score=rmse
    print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


----------------------------------- 0 ----------------------------------------

Lasso score: 0.3592 (0.0205)

----------------------------------- 1 ----------------------------------------

Lasso score: 0.3592 (0.0205)

----------------------------------- 2 ----------------------------------------

Lasso score: 0.3592 (0.0205)

----------------------------------- 3 ----------------------------------------

Lasso score: 0.3592 (0.0205)

----------------------------------- 4 ----------------------------------------

Lasso score: 0.3592 (0.0205)

----------------------------------- 5 ----------------------------------------

Lasso score: 0.3592 (0.0205)

----------------------------------- 6 ----------------------------------------

Lasso score: 0.3592 (0.0205)

----------------------------------- 7 ----------------------------------------

Lasso score: 0.3592 (0.0205)

----------------------------------- 8 ----------------------------------------

Lasso score: 0.3592 (0.0205)



In [96]:
n_folds=5
for i in range(9):
    print('-----------------------------------',i,'----------------------------------------')
    lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x1[np.where(y1>2)[0]])
    rmse=np.sqrt(-cross_val_score(lasso,x1[np.where(y1>2)[0]],y1[np.where(y1>2)[0]],scoring='neg_mean_squared_error',cv=kf))
    score=rmse
    print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

----------------------------------- 0 ----------------------------------------

Lasso score: 0.6557 (0.0407)

----------------------------------- 1 ----------------------------------------

Lasso score: 0.6557 (0.0407)

----------------------------------- 2 ----------------------------------------

Lasso score: 0.6557 (0.0407)

----------------------------------- 3 ----------------------------------------

Lasso score: 0.6557 (0.0407)

----------------------------------- 4 ----------------------------------------

Lasso score: 0.6557 (0.0407)

----------------------------------- 5 ----------------------------------------

Lasso score: 0.6557 (0.0407)

----------------------------------- 6 ----------------------------------------

Lasso score: 0.6557 (0.0407)

----------------------------------- 7 ----------------------------------------

Lasso score: 0.6557 (0.0407)

----------------------------------- 8 ----------------------------------------

Lasso score: 0.6557 (0.0407)



In [139]:
np.where(y>5)

(array([   0,    1,    2, ..., 5745, 5746, 5747], dtype=int64),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

In [140]:
y1[50:60]

array([[4.47213595],
       [4.        ],
       [3.74165739],
       [3.87298335],
       [2.82842712],
       [2.        ],
       [3.        ],
       [4.        ],
       [4.69041576],
       [4.79583152]])

In [141]:
y[50:60]

array([[20.],
       [16.],
       [14.],
       [15.],
       [ 8.],
       [ 4.],
       [ 9.],
       [16.],
       [22.],
       [23.]])

In [146]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=7)

In [147]:
x_train.shape

(4598, 18)

In [148]:
y_train.shape

(4598, 1)

In [149]:
n_folds=5
print('-----------------------------------','start','----------------------------------------')
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x_train)
rmse=np.sqrt(-cross_val_score(lasso,x_train,np.sqrt(y_train),scoring='neg_mean_squared_error',cv=kf))
score=rmse
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

----------------------------------- start ----------------------------------------

Lasso score: 0.7400 (0.0105)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [154]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))
lasso.fit(x_train,np.sqrt(y_train))
y_test_hat=lasso.predict(x_test)
np.sqrt(mean_squared_error(np.sqrt(y_test),y_test_hat))

0.7736902134062523

In [156]:
np.sqrt(mean_squared_error(y_test,np.square(y_test_hat)))

7.442349091803772

In [161]:
for i in range(len(y_test)):
    print(y_test[i],y_test_hat[i],np.square(y_test_hat[i]),np.abs(y_test[i]-np.square(y_test_hat[i])))

[8.] 2.929743775285658 8.58339858882506 [0.58339859]
[25.] 4.86331904052096 23.651872089893708 [1.34812791]
[10.] 3.038975531362565 9.235372280220384 [0.76462772]
[34.] 5.97163914896281 35.660474125425274 [1.66047413]
[3.] 2.4382631524070018 5.94512720038573 [2.9451272]
[66.] 8.808456795316541 77.58891111495815 [11.58891111]
[14.] 4.2112858911740565 17.734928857201666 [3.73492886]
[27.] 4.22047364207754 17.812397763471253 [9.18760224]
[34.] 5.4145780046672805 29.31765496862671 [4.68234503]
[27.] 4.692654046646057 22.021002001503614 [4.978998]
[24.] 4.243924428810375 18.010894557453465 [5.98910544]
[9.] 3.0352636254397263 9.212825275917512 [0.21282528]
[10.] 3.127672096372257 9.782332742425629 [0.21766726]
[8.] 3.203271512655244 10.260948383788616 [2.26094838]
[12.] 3.081824586270629 9.497642780542133 [2.50235722]
[41.] 7.537932755617192 56.82043022820659 [15.82043023]
[25.] 8.42618235170818 71.0005490242384 [46.00054902]
[12.] 3.0555115933540735 9.33615109712115 [2.6638489]
[11.] 3.043

[15.] 3.8709718651846674 14.984423181051262 [0.01557682]
[12.] 3.9143599026824414 15.322213447728092 [3.32221345]
[21.] 3.3819460823234238 11.437559303742754 [9.5624407]
[13.] 4.356344329472006 18.977735916922903 [5.97773592]
[3.] 2.47520771418266 6.126653228349348 [3.12665323]
[26.] 4.412806943733232 19.472865122660227 [6.52713488]
[1.] 2.2137714778269446 4.900784156040094 [3.90078416]
[26.] 4.513775596228348 20.37417013310658 [5.62582987]
[33.] 4.782762551404251 22.8748176231149 [10.12518238]
[26.] 4.444110139496779 19.75011493197808 [6.24988507]
[2.] 2.3603450495538536 5.571228752953384 [3.57122875]
[36.] 4.452521794435507 19.824950329923183 [16.17504967]
[8.] 2.9478527725191066 8.689835968448584 [0.68983597]
[17.] 4.39582816096932 19.323305220770916 [2.32330522]
[23.] 4.491769338147929 20.175991787125884 [2.82400821]
[20.] 5.012306615915192 25.123217611947208 [5.12321761]
[24.] 5.067680261814516 25.68138323598444 [1.68138324]
[2.] 2.4512267291257457 6.008512477580502 [4.00851248]
[

[18.] 3.6806677868638142 13.547315357256968 [4.45268464]
[22.] 4.15266924333551 17.24466184454472 [4.75533816]
[22.] 4.906529494574105 24.074031681125625 [2.07403168]
[29.] 4.8214441614895795 23.246323802361953 [5.7536762]
[49.] 5.664944187541984 32.09159264796571 [16.90840735]
[38.] 5.9172768977388195 35.014165884513545 [2.98583412]
[20.] 4.104836853489087 16.84968559376219 [3.15031441]
[25.] 4.187705710211982 17.53687911534204 [7.46312088]
[22.] 3.715437155026503 13.804473252951434 [8.19552675]
[6.] 3.7116923143260956 13.776659836227408 [7.77665984]
[24.] 4.226000472549259 17.859079993986562 [6.14092001]
[17.] 3.5209978496808754 12.397425857457348 [4.60257414]
[25.] 5.333621803086925 28.44752153836422 [3.44752154]
[21.] 4.198485713412922 17.627282285732417 [3.37271771]
[18.] 5.163278826600127 26.659448241217184 [8.65944824]
[25.] 4.223237705394327 17.835736716264343 [7.16426328]
[13.] 3.6265294774687487 13.151716050949755 [0.15171605]
[11.] 3.4516858057806252 11.914134901827444 [0.91

In [160]:
for i in y_test_hat:
    print(i)

2.929743775285658
4.86331904052096
3.038975531362565
5.97163914896281
2.4382631524070018
8.808456795316541
4.2112858911740565
4.22047364207754
5.4145780046672805
4.692654046646057
4.243924428810375
3.0352636254397263
3.127672096372257
3.203271512655244
3.081824586270629
7.537932755617192
8.42618235170818
3.0555115933540735
3.043356348884508
4.598068224777162
4.689977230568493
3.790462092051256
4.787016217562689
2.2433555843665367
2.406388873573227
3.8432913257465686
3.6774830128302805
3.9053343222511145
5.560563234168358
3.4558846512438963
5.081530503672931
7.0669761330255145
3.0317411206948752
4.865504481879315
2.9092606309234514
2.8599559119397697
5.74616927495592
2.9896061779324405
3.790678546954463
8.632519553229722
3.0447944530370488
5.551192558375005
2.9613453804143663
3.2593491004604536
7.466435510590131
4.051993597679458
5.5693676411823665
2.7018062417251025
6.281940106392837
4.683543559889095
7.6553304490425305
2.4736418112535743
5.605748893464078
3.8113006230258826
2.90502280

In [167]:
plt.plot(y)
plt.show()

In [168]:
def train_k_fold_data(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.MSELoss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    train_l,test_l=[],[]
    
    min_test_loss=1000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#         if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        with torch.no_grad():
            test_loss=loss(net(test_features),test_labels).item()
            if test_loss<min_test_loss:
                min_test_loss=test_loss
#                 test_l.append(test_loss)
                train_loss=loss(net(train_features),train_labels).item()
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l

In [169]:
def get_kfold_data(k,j,x,y,random_state=13):
    assert k>=1, 'k must >=1'
    fold_size = x.shape[0] // k
    x_train,y_train=None,None
    row_list=list(range(x.shape[0]))
    np.random.seed(random_state)
    np.random.shuffle(row_list)
    for i in range(k):
        idx=slice(fold_size*i,fold_size*(i+1))
        x_part,y_part=x[row_list[idx],:],y[row_list[idx],:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val

In [170]:
def train_kfold(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')

In [ ]:
def get_net_lstm():
    return nn.Sequential(nn.Linear(x1.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x1,y1,256,0,0)

In [ ]:
nn.B